# Day 3: Simple Web Chatbot (25 minutes)
## Build a Web Interface with Gradio

Today we'll create a simple web interface for our RAG chatbot using Gradio!

In [1]:
# Quick setup
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package.split('==')[0])
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install packages
packages = ["gradio", "sentence-transformers", "scikit-learn", "pandas"]
for pkg in packages:
    install_if_needed(pkg)

print("Setup complete!")

In [2]:
# Import and setup (rebuild the chatbot from Day 2)
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Load data and model
df = pd.read_csv(r"C:\Users\vHaze\Desktop\LLMinds\workshop2\data\simple_faq.csv")
model = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = model.encode(df['question'].tolist())

print(f"Loaded {len(df)} FAQs and embedding model")

In [ ]:
# Create a reference vector for the word "Hello" to check for greetings.
# This is done once at startup for performance, so the bot doesn't have to
# re-calculate it for every single message. Any user input that is
# semantically similar to this vector will be treated as a greeting.

GREETING_VECTOR = model.encode(["Hello"])[0]
GREETING_THRESHOLD = 0.5 
print("Greeting vector created and ready.")

In [5]:
# Simple chatbot function for Gradio with semantic greeting logic
def chatbot_response(user_question, threshold=0.3):
    if not user_question.strip():
        return "Please ask a question!"
    
    # --- Semantic Greeting Logic Added ---
    # Encode the user's question once for efficiency
    query_embedding = model.encode([user_question])[0] # Get the 1D vector

    # Check for greeting similarity against the pre-computed vector
    greeting_similarity = cosine_similarity([query_embedding], [GREETING_VECTOR])[0][0]
    
    if greeting_similarity > GREETING_THRESHOLD:
        return "Hello! How can I help you today? 😊"

    # --- FAQ Search Logic (reuses the embedding) ---
    # Find most similar FAQ using the same embedding
    similarities = cosine_similarity([query_embedding], question_embeddings)[0] # Note: [query_embedding] makes it 2D
    best_match_idx = np.argmax(similarities)
    best_similarity = similarities[best_match_idx]
    
    # Generate response
    if best_similarity < threshold:
        response = "I don't have information about that. Please contact our support team!"
    else:
        response = df.iloc[best_match_idx]['answer']
        confidence = "High" if best_similarity > 0.7 else "Medium"
        matched_question = df.iloc[best_match_idx]['question']
        category = df.iloc[best_match_idx]['category']
        
        # Add context info
        response += f"\n\n📊 **Confidence:** {confidence} ({best_similarity:.3f})"
        response += f"\n📂 **Category:** {category}"
        response += f"\n**🔍 Matched:** {matched_question}"
    
    return response

# Test the function with a question and a greeting
print("--- Testing FAQ ---")
print(chatbot_response("How can I pay for my order?"))
print("\n--- Testing Greeting ---")
print(chatbot_response("Hey what's up"))

In [6]:
# Create Gradio interface
def create_chatbot_interface():
    # Create the interface
    iface = gr.Interface(
        fn=chatbot_response,
        inputs=[
            gr.Textbox(
                label="Ask your question",
                placeholder="Type your question here... (e.g., 'What are your business hours?')",
                lines=2
            )
        ],
        outputs=gr.Textbox(
            label="Chatbot Response",
            lines=6
        ),
        title="🤖 Simple FAQ Chatbot",
        description="Ask questions about our services! Try asking about hours, support, payment, shipping, etc.",
        examples=[
            ["What are your business hours?"],
            ["How can I contact support?"],
            ["What payment methods do you accept?"],
            ["How long does shipping take?"],
            ["Can I return my order?"],
        ],
        theme=gr.themes.Soft(),
        allow_flagging="never"
    )
    
    return iface

# Create the interface
chatbot_interface = create_chatbot_interface()
print("Gradio interface created!")

In [7]:
# Launch the web interface
print("Launching chatbot web interface...")
print("This will open in your browser!")

# Launch with share=True to get a public link
chatbot_interface.launch(
    share=True,      # Creates public link
    server_port=7860,  # Port number
    show_error=True    # Show errors in interface
)

print("\n🎉 Congratulations! You've built a complete RAG chatbot with web interface!")
print("\n📝 What you've learned:")
print("- Loading and analyzing CSV data")
print("- Using Hugging Face embeddings")
print("- Building RAG (Retrieval-Augmented Generation)")
print("- Creating web interfaces with Gradio")
print("\n🚀 Next steps: Add more FAQs, try different models, improve the interface!")